# Working with DataFrames

This is part two of a three part introduction to pandas, a Python library for data analysis. The tutorial is primarily geared towards SQL users, but is useful for anyone wanting to get started with the library.

Part 1: Intro to pandas data structures

Part 2: Working with DataFrames

Part 3: Using pandas with the MovieLens dataset

## Working with DataFrames

Now that we can get data into a DataFrame, we can finally start working with them. pandas has an abundance of functionality, far too much for me to cover in this introduction. I'd encourage anyone interested in diving deeper into the library to check out its excellent documentation. Or just use Google - there are a lot of Stack Overflow questions and blog posts covering specifics of the library.

We'll be using the MovieLens dataset in many examples going forward. The dataset contains 100,000 ratings made by 943 users on 1,682 movies.

In [10]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
pd.set_option('max_columns', 50)
%matplotlib inline
engine = 'python'
warnings.simplefilter("ignore")

In [11]:
# pass in column names for each CSV
u_cols = ['user_id', 'sex', 'age', 'occupation', 'zip_code']
users = pd.read_csv('/Users/markyashar//python_data_analysis/pydata-book/pydata-notebook-master/ch02/movielens/users.dat',
                    sep='::', names=u_cols,
                    encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('/Users/markyashar//python_data_analysis/pydata-book/pydata-notebook-master/ch02/movielens/users.dat'
                      , sep='::', names=r_cols,
                      encoding='latin-1')

# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title and release_date', 'genre']
movies = pd.read_csv('/Users/markyashar//python_data_analysis/pydata-book/pydata-notebook-master/ch02/movielens/movies.dat',
                     sep='::', names=m_cols, usecols=range(3), encoding='latin-l')
print(users.head())
print(ratings.head())
print(movies.head())

   user_id sex  age  occupation zip_code
0        1   F    1          10    48067
1        2   M   56          16    70072
2        3   M   25          15    55117
3        4   M   45           7    02460
4        5   M   25          20    55455
  user_id  movie_id  rating unix_timestamp
1       F         1      10          48067
2       M        56      16          70072
3       M        25      15          55117
4       M        45       7          02460
5       M        25      20          55455
   movie_id              title and release_date                         genre
0         1                    Toy Story (1995)   Animation|Children's|Comedy
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy
2         3             Grumpier Old Men (1995)                Comedy|Romance
3         4            Waiting to Exhale (1995)                  Comedy|Drama
4         5  Father of the Bride Part II (1995)                        Comedy


## Inspection

pandas has a variety of functions for getting basic information about your DataFrame, the most basic of which is using the **info** method.

In [12]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
movie_id                  3883 non-null int64
title and release_date    3883 non-null object
genre                     3883 non-null object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


The output tells a few things about our DataFrame.

1. It's obviously an instance of a DataFrame.
2. Each row was assigned an index of 0 to N-1, where N is the number of rows in the DataFrame. pandas will do this by default if an index is not specified. Don't worry, this can be changed later.
3. There are 3,883 rows (every row must have an index).
4. Our dataset has three total columns...
5. The last datatypes of each column, but not necessarily in the corresponding order to the listed columns. You should use the dtypes method to get the datatype for each column.
6. An approximate amount of RAM used to hold the DataFrame. See the **.memory_usage method**


In [13]:
movies.dtypes

movie_id                   int64
title and release_date    object
genre                     object
dtype: object

DataFrame's also have a **describe** method, which is great for seeing basic statistics about the dataset's numeric columns. Be careful though, since this will return information on all columns of a numeric datatype.

In [17]:
users.describe()

,user_id,age,occupation
count,6040.000000,6040.000000,6040.000000
mean,3020.500000,30.639238,8.146854
std,1743.742145,12.895962,6.329511
min,1.000000,1.000000,0.000000
25%,1510.750000,25.000000,3.000000
50%,3020.500000,25.000000,7.000000
75%,4530.250000,35.000000,14.000000
max,6040.000000,56.000000,20.000000


Notice **user_id** was included since it's numeric. Since this is an ID value, the stats for it don't really matter.

We can quickly see the average age of our users is just above 31 years old, with the youngest being 1 and the oldest being 56. The median age is 25, with the youngest quartile of users being 25 or younger, and the oldest quartile being at least 35.

You've probably noticed that I've used the head method regularly throughout this post - by default, head displays the first five records of the dataset, while tail displays the last five.

In [18]:
movies.head()

,movie_id,title and release_date,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [19]:
movies.tail(3)

,movie_id,title and release_date,genre
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


Alternatively, Python's regular **slicing** syntax works as well.

In [22]:
movies[20:22]

,movie_id,title and release_date,genre
20,21,Get Shorty (1995),Action|Comedy|Drama
21,22,Copycat (1995),Crime|Drama|Thriller


## Selecting

You can think of a DataFrame as a group of Series that share an index (in this case the column headers). This makes it easy to select specific columns.

Selecting a single column from the DataFrame will return a Series object.

In [21]:
users['occupation'].head()   # of course, this really isn't occupation -- we're just testing things out here

0    10
1    16
2    15
3     7
4    20
Name: occupation, dtype: int64

## To select multiple columns, simply pass a list of column names to the DataFrame, the output of which will be a DataFrame.

In [23]:
print(users[['age', 'zip_code']].head())
print('\n')

# can also store in a variable to use later
columns_you_want = ['occupation', 'sex'] 
print(users[columns_you_want].head())

   age zip_code
0    1    48067
1   56    70072
2   25    55117
3   45    02460
4   25    55455


   occupation sex
0          10   F
1          16   M
2          15   M
3           7   M
4          20   M


Row selection can be done multiple ways, but doing so by an individual index or boolean indexing are typically easiest.



In [24]:
# users older than 25
print(users[users.age > 25].head(3))
print('\n')

# users aged 40 AND male
print(users[(users.age == 40) & (users.sex == 'M')].head(3))
print('\n')

# users younger than 30 OR female
print(users[(users.sex == 'F') | (users.age < 30)].head(3))

   user_id sex  age  occupation zip_code
1        2   M   56          16    70072
3        4   M   45           7    02460
5        6   F   50           9    55117


Empty DataFrame
Columns: [user_id, sex, age, occupation, zip_code]
Index: []


   user_id sex  age  occupation zip_code
0        1   F    1          10    48067
2        3   M   25          15    55117
4        5   M   25          20    55455


Since our index is kind of meaningless right now, let's set it to the user_id using the set_index method. By default, **set_index** returns a new DataFrame, so you'll have to specify if you'd like the changes to occur in place.

This has confused me in the past, so look carefully at the code and output below.

In [25]:
print(users.set_index('user_id').head())
print('\n')

print(users.head())
print("\n^^^ I didn't actually change the DataFrame. ^^^\n")

with_new_index = users.set_index('user_id')
print(with_new_index.head())
print("\n^^^ set_index actually returns a new DataFrame. ^^^\n")

        sex  age  occupation zip_code
user_id                              
1         F    1          10    48067
2         M   56          16    70072
3         M   25          15    55117
4         M   45           7    02460
5         M   25          20    55455


   user_id sex  age  occupation zip_code
0        1   F    1          10    48067
1        2   M   56          16    70072
2        3   M   25          15    55117
3        4   M   45           7    02460
4        5   M   25          20    55455

^^^ I didn't actually change the DataFrame. ^^^

        sex  age  occupation zip_code
user_id                              
1         F    1          10    48067
2         M   56          16    70072
3         M   25          15    55117
4         M   45           7    02460
5         M   25          20    55455

^^^ set_index actually returns a new DataFrame. ^^^



If you want to modify your existing DataFrame, use the **inplace** parameter. Most DataFrame methods return new a DataFrames, while offering an **inplace** parameter. Note that the inplace version might not actually be any more efficint (in terms of speed or memory usage) that the regular version.



In [26]:
users.set_index('user_id', inplace=True)
users.head()

,sex,age,occupation,zip_code
user_id,,,,
1,F,1,10,48067
2,M,56,16,70072
3,M,25,15,55117
4,M,45,7,02460
5,M,25,20,55455


Notice that we've lost the default pandas 0-based index and moved the user_id into its place. We can select rows **by position** using the **iloc** method.

In [27]:
print(users.iloc[99])
print('\n')
print(users.iloc[[1, 50, 300]])

sex               M
age              35
occupation       17
zip_code      95401
Name: 100, dtype: object


        sex  age  occupation zip_code
user_id                              
2         M   56          16    70072
51        F    1          10    10562
301       M   18           4    61820


And we can select rows **by label** with the **loc** method.

In [29]:
print(users.loc[100])
print('\n')
print(users.loc[[2, 51, 100, 301]])

sex               M
age              35
occupation       17
zip_code      95401
Name: 100, dtype: object


        sex  age  occupation zip_code
user_id                              
2         M   56          16    70072
51        F    1          10    10562
100       M   35          17    95401
301       M   18           4    61820


If we realize later that we liked the old pandas default index, we can just **reset_index**. The same rules for **inplace** apply.

In [30]:
users.reset_index(inplace=True)
users.head()

,user_id,sex,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


The simplified rules of indexing are

Use **loc** for label-based indexing
Use **iloc** for positional indexing
I've found that I can usually get by with boolean indexing, loc and iloc, but pandas has a whole host of **other ways to do selection**.

## Joining

Throughout an analysis, we'll often need to merge/join datasets as data is typically stored in a relational manner.

Our MovieLens data is a good example of this - a rating requires both a user and a movie, and the datasets are linked together by a key - in this case, the user_id and movie_id. It's possible for a user to be associated with zero or many ratings and movies. Likewise, a movie can be rated zero or many times, by a number of different users.

Like SQL's JOIN clause, **pandas.merge** allows two DataFrames to be joined on one or more keys. The function provides a series of parameters (**on, left_on, right_on, left_index, right_index**) allowing you to specify the columns or indexes on which to join.

By default, **pandas.merge** operates as an inner join, which can be changed using the how parameter.

From the function's docstring:

**how**: {'left', 'right', 'outer', 'inner'}, default 'inner'

**left**: use only keys from left frame (SQL: left outer join)

**right**: use only keys from right frame (SQL: right outer join)

**outer**: use union of keys from both frames (SQL: full outer join)

**inner**: use intersection of keys from both frames (SQL: inner join)

Below are some examples of what each look like.

In [31]:
left_frame = pd.DataFrame({'key': range(5), 
                           'left_value': ['a', 'b', 'c', 'd', 'e']})
right_frame = pd.DataFrame({'key': range(2, 7), 
                           'right_value': ['f', 'g', 'h', 'i', 'j']})
print(left_frame)
print('\n')
print(right_frame)


   key left_value
0    0          a
1    1          b
2    2          c
3    3          d
4    4          e


   key right_value
0    2           f
1    3           g
2    4           h
3    5           i
4    6           j


## inner join (default)

In [32]:
pd.merge(left_frame, right_frame, on='key', how='inner')

,key,left_value,right_value
0,2,c,f
1,3,d,g
2,4,e,h


We lose values from both frames since certain keys do not match up. The SQL equivalent is:

    SELECT left_frame.key, left_frame.left_value, right_frame.right_value
    FROM left_frame
    INNER JOIN right_frame
        ON left_frame.key = right_frame.key;
        
Had our key columns not been named the same, we could have used the **left_on** and **right_on** parameters to specify which fields to join from each frame. python     **pd.merge(left_frame, right_frame, left_on='left_key', right_on='right_key')** Alternatively, if our keys were indexes, we could use the **left_index** or **right_index** parameters, which accept a True/False value. You can mix and match columns and indexes like so: python     **pd.merge(left_frame, right_frame, left_on='key', right_index=True)**



## left outer join

In [33]:
pd.merge(left_frame, right_frame, on='key', how='left')

,key,left_value,right_value
0,0,a,NaN
1,1,b,NaN
2,2,c,f
3,3,d,g
4,4,e,h


We keep everything from the left frame, pulling in the value from the right frame where the keys match up. The **right_value** is NULL where keys do not match (NaN).

SQL Equivalent:

  SELECT left_frame.key, left_frame.left_value, right_frame.right_value
  
  FROM left_frame
  
  LEFT JOIN right_frame
  
      ON left_frame.key = right_frame.key;
    

## right outer join

In [34]:
pd.merge(left_frame, right_frame, on='key', how='right')

,key,left_value,right_value
0,2.0,c,f
1,3.0,d,g
2,4.0,e,h
3,5.0,NaN,i
4,6.0,NaN,j


This time we've kept everything from the right frame with the **left_value** being NULL where the right frame's key did not find a match.

SQL Equivalent:

SELECT right_frame.key, left_frame.left_value, right_frame.right_value

FROM left_frame

RIGHT JOIN right_frame

    ON left_frame.key = right_frame.key;


## full outer join

In [35]:
pd.merge(left_frame, right_frame, on='key', how='outer')

,key,left_value,right_value
0,0.0,a,NaN
1,1.0,b,NaN
2,2.0,c,f
3,3.0,d,g
4,4.0,e,h
5,5.0,NaN,i
6,6.0,NaN,j


We've kept everything from both frames, regardless of whether or not there was a match on both sides. Where there was not a match, the values corresponding to that key are NULL.

SQL Equivalent (though some databases don't allow FULL JOINs (e.g. MySQL)):

SELECT IFNULL(left_frame.key, right_frame.key) key

        , left_frame.left_value, right_frame.right_value
        
FROM left_frame

FULL OUTER JOIN right_frame

    ON left_frame.key = right_frame.key;


## Combining

pandas also provides a way to combine DataFrames along an axis - **pandas.concat**. While the function is equivalent to SQL's UNION clause, there's a lot more that can be done with it.

**pandas.concat** takes a list of Series or DataFrames and returns a Series or DataFrame of the concatenated objects. Note that because the function takes list, you can combine many objects at once.

In [36]:
pd.concat([left_frame, right_frame])

,key,left_value,right_value
0,0,a,NaN
1,1,b,NaN
2,2,c,NaN
3,3,d,NaN
4,4,e,NaN
0,2,NaN,f
1,3,NaN,g
2,4,NaN,h
3,5,NaN,i
4,6,NaN,j


By default, the function will vertically append the objects to one another, combining columns with the same name. We can see above that values not matching up will be NULL.

Additionally, objects can be concatentated side-by-side using the function's **axis** parameter.

In [37]:
pd.concat([left_frame, right_frame], axis=1)

# Signature: pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None,
#                      verify_integrity=False, copy=True)
# Docstring:
# Concatenate pandas objects along a particular axis with optional set logic
# along the other axes. Can also add a layer of hierarchical indexing on the
# concatenation axis, which may be useful if the labels are the same (or
# overlapping) on the passed axis number

# Parameters
# ----------
# objs : a sequence or mapping of Series, DataFrame, or Panel objects
#     If a dict is passed, the sorted keys will be used as the `keys`
#     argument, unless it is passed, in which case the values will be
#     selected (see below). Any None objects will be dropped silently unless
#     they are all None in which case a ValueError will be raised
# axis : {0, 1, ...}, default 0
#     The axis to concatenate along
# join : {'inner', 'outer'}, default 'outer'
#     How to handle indexes on other axis(es)
# join_axes : list of Index objects
#     Specific indexes to use for the other n - 1 axes instead of performing
#     inner/outer set logic
# verify_integrity : boolean, default False
#     Check whether the new concatenated axis contains duplicates. This can
#     be very expensive relative to the actual data concatenation
# keys : sequence, default None
#     If multiple levels passed, should contain tuples. Construct
#     hierarchical index using the passed keys as the outermost level
# levels : list of sequences, default None
#     Specific levels (unique values) to use for constructing a
#     MultiIndex. Otherwise they will be inferred from the keys
# names : list, default None
#     Names for the levels in the resulting hierarchical index
# ignore_index : boolean, default False
#     If True, do not use the index values along the concatenation axis. The
#     resulting axis will be labeled 0, ..., n - 1. This is useful if you are
#     concatenating objects where the concatenation axis does not have
#     meaningful indexing information. Note the index values on the other
#     axes are still respected in the join.
# copy : boolean, default True
#     If False, do not copy data unnecessarily

# Notes
# -----
# The keys, levels, and names arguments are all optional

# Returns
# -------
# concatenated : type of objects

,key,left_value,key,right_value
0,0,a,2,f
1,1,b,3,g
2,2,c,4,h
3,3,d,5,i
4,4,e,6,j


**pandas.concat** can be used in a variety of ways; however, we've typically only used it to combine Series/DataFrames into one unified object. The documentation has some examples on the ways it can be used.

## Grouping

Grouping in pandas took some time for me to grasp, but it's pretty awesome once it clicks.

pandas **groupby** method draws largely from the split-apply-combine strategy for data analysis. If you're not familiar with this methodology, I highly suggest you read up on it. It does a great job of illustrating how to properly think through a data problem, which I feel is more important than any technical skill a data analyst/scientist can possess.

When approaching a data analysis problem, you'll often break it apart into manageable pieces, perform some operations on each of the pieces, and then put everything back together again (this is the gist split-apply-combine strategy). pandas **groupby** is great for these problems (R users should check out the plyr and dplyr packages).

If you've ever used SQL's GROUP BY or an Excel Pivot Table, you've thought with this mindset, probably without realizing it.

Assume we have a DataFrame and want to get the average for each group - visually, the split-apply-combine method looks like this:  http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/

The City of Chicago is kind enough to publish all city employee salaries to its open data portal. Let's go through some basic groupby examples using this data.

In [38]:
from_csv = pd.read_csv('https://raw.githubusercontent.com/gjreda/gregreda.com/master/content/notebooks/data/city-of-chicago-salaries.csv')
from_csv.head()

,Name,Position Title,Department,Employee Annual Salary
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT,$85512.00
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE,$75372.00
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,$80916.00
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,$99648.00
4,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,$89440.00


Since the data contains a dollar sign for each salary, python will treat the field as a series of strings. We can use the converters parameter to change this when reading in the file.

converters : dict. optional

 * Dict of functions for converting values in certain columns. Keys can either be integers or column labels

In [39]:
headers = ['name', 'title', 'department', 'salary']
chicago = pd.read_csv('https://raw.githubusercontent.com/gjreda/gregreda.com/master/content/notebooks/data/city-of-chicago-salaries.csv', 
                      header=0,
                      names=headers,
                      converters={'salary': lambda x: float(x.replace('$', ''))})
chicago.head()

,name,title,department,salary
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT,85512.0
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE,75372.0
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,80916.0
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,99648.0
4,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,89440.0


pandas **groupby** returns a DataFrameGroupBy object which has a variety of methods, many of which are similar to standard SQL aggregate functions.

In [40]:
by_dept = chicago.groupby('department')
by_dept

Calling **count** returns the total number of NOT NULL values within each column. If we were interested in the total number of records in each group, we could use **size**.

In [41]:
print(by_dept.count().head()) # NOT NULL records within each column
print('\n')
print(by_dept.size().tail()) # total records for each department

                   name  title  salary
department                            
ADMIN HEARNG         42     42      42
ANIMAL CONTRL        61     61      61
AVIATION           1218   1218    1218
BOARD OF ELECTION   110    110     110
BOARD OF ETHICS       9      9       9


department
PUBLIC LIBRARY     926
STREETS & SAN     2070
TRANSPORTN        1168
TREASURER           25
WATER MGMNT       1857
dtype: int64


Summation can be done via **sum**, averaging by **mean**, etc. (if it's a SQL function, chances are it exists in pandas). Oh, and there's median too, something not available in most databases.

In [42]:
print(by_dept.sum()[20:25]) # total salaries of each department
print('\n')
print(by_dept.mean()[20:25]) # average salary of each department
print('\n')
print(by_dept.median()[20:25]) # take that, RDBMS!

                       salary
department                   
HUMAN RESOURCES     4850928.0
INSPECTOR GEN       4035150.0
IPRA                7006128.0
LAW                31883920.2
LICENSE APPL COMM     65436.0


                         salary
department                     
HUMAN RESOURCES    71337.176471
INSPECTOR GEN      80703.000000
IPRA               82425.035294
LAW                70853.156000
LICENSE APPL COMM  65436.000000


                    salary
department                
HUMAN RESOURCES    68496.0
INSPECTOR GEN      76116.0
IPRA               82524.0
LAW                66492.0
LICENSE APPL COMM  65436.0


Operations can also be done on an individual Series within a grouped object. Say we were curious about the five departments with the most distinct titles - the pandas equivalent to:

SELECT department, COUNT(DISTINCT title)

FROM chicago

GROUP BY department

ORDER BY 2 DESC

LIMIT 5;

pandas is a lot less verbose here ...

In [43]:
by_dept.title.nunique().sort_values(ascending=False)[:5]

department
WATER MGMNT    153
TRANSPORTN     150
POLICE         130
AVIATION       125
HEALTH         118
Name: title, dtype: int64

## split-apply-combine

The real power of **groupby** comes from it's split-apply-combine ability.

What if we wanted to see the highest paid employee within each department. Given our current dataset, we'd have to do something like this in SQL:

SELECT *
FROM chicago c
INNER JOIN (
    SELECT department, max(salary) max_salary
    FROM chicago
    GROUP BY department
) m
ON c.department = m.department
AND c.salary = m.max_salary;

This would give you the highest paid person in each department, but it would return multiple if there were many equally high paid people within a department.

Alternatively, you could alter the table, add a column, and then write an update statement to populate that column. However, that's not always an option.

Note: This would be a lot easier in PostgreSQL, T-SQL, and possibly Oracle due to the existence of partition/window/analytic functions. I've chosen to use MySQL syntax throughout this tutorial because of it's popularity. Unfortunately, MySQL doesn't have similar functions.

Using **groupby** we can define a function (which we'll call **ranker**) that will label each record from 1 to N, where N is the number of employees within the department. We can then call apply to, well, **apply** that function to each group (in this case, each department).

In [44]:
def ranker(df):
    """Assigns a rank to each employee based on salary, with 1 being the highest paid.
    Assumes the data is DESC sorted."""
    df['dept_rank'] = np.arange(len(df)) + 1
    return df


In [46]:
chicago.sort_values('salary', ascending=False, inplace=True)

# Signature: chicago.sort_values(by, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# Docstring:
# Sort by the values along either axis

# Parameters
# ----------
# by : string name or list of names which refer to the axis items
# axis : index, columns to direct sorting
# ascending : bool or list of bool
#      Sort ascending vs. descending. Specify list for multiple sort
#      orders.  If this is a list of bools, must match the length of
#      the by.
# inplace : bool
#      if True, perform operation in-place
# kind : {`quicksort`, `mergesort`, `heapsort`}
#      Choice of sorting algorithm. See also ndarray.np.sort for more
#      information.  `mergesort` is the only stable algorithm. For
#      DataFrames, this option is only applied when sorting on a single
#      column or label.
# na_position : {'first', 'last'}
#      `first` puts NaNs at the beginning, `last` puts NaNs at the end

# Returns
# -------
# sorted_obj : DataFrame

chicago = chicago.groupby('department').apply(ranker)
print(chicago[chicago.dept_rank == 1].head(7))
chicago.head()

                         name                     title      department  \
18039     MC CARTHY,  GARRY F  SUPERINTENDENT OF POLICE          POLICE   
8004           EMANUEL,  RAHM                     MAYOR  MAYOR'S OFFICE   
25588       SANTIAGO,  JOSE A         FIRE COMMISSIONER            FIRE   
763    ANDOLINO,  ROSEMARIE S  COMMISSIONER OF AVIATION        AVIATION   
4697     CHOUCAIR,  BECHARA N    COMMISSIONER OF HEALTH          HEALTH   
21971      PATTON,  STEPHEN R       CORPORATION COUNSEL             LAW   
12635      HOLT,  ALEXANDRA D                BUDGET DIR   BUDGET & MGMT   

         salary  dept_rank  
18039  260004.0          1  
8004   216210.0          1  
25588  202728.0          1  
763    186576.0          1  
4697   177156.0          1  
21971  173664.0          1  
12635  169992.0          1  


,name,title,department,salary,dept_rank
18039,"MC CARTHY, GARRY F",SUPERINTENDENT OF POLICE,POLICE,260004.0,1
8004,"EMANUEL, RAHM",MAYOR,MAYOR'S OFFICE,216210.0,1
25588,"SANTIAGO, JOSE A",FIRE COMMISSIONER,FIRE,202728.0,1
27594,"STEWART III, CHARLES",FIRST DEPUTY FIRE COMMISSIONER,FIRE,188316.0,2
31587,"WYSINGER, ALFONZA",FIRST DEPUTY SUPERINTENDENT,POLICE,188316.0,2


In [47]:
chicago[chicago.department == "LAW"][:5]

,name,title,department,salary,dept_rank
21971,"PATTON, STEPHEN R",CORPORATION COUNSEL,LAW,173664.0,1
6311,"DARLING, LESLIE M",FIRST ASST CORPORATION COUNSEL,LAW,149160.0,2
17680,"MARTINICO, JOSEPH P",CHIEF LABOR NEGOTIATOR,LAW,144036.0,3
22357,"PETERS, LYNDA A",CITY PROSECUTOR,LAW,139932.0,4
7620,"DUNN, JAMES L",ADMINISTRATIVE DEPUTY,LAW,137076.0,5


We can now see where each employee ranks within their department based on salary.